In [ ]:
import json

In [ ]:
with open("/mnt/disk6/Alfred/Rescoring/data/aishell2/data/withLM/train/data.json") as f:
    gen_data = json.load(f)

In [ ]:
with open("/mnt/disk6/Alfred/Rescoring/data/aishell2/raw/withLM/train/data.json") as f:
    data = json.load(f)

In [ ]:
print(type(data))

In [ ]:
print(type(gen_data))

In [ ]:
count =0
s = []
for d in data['utts'].keys():
    if (len(data['utts'][d]['output'])) <= 1:
        count += 1
        s.append(d)

print(f'{count}')
for a in s:
    print(f"{a}\n")

In [ ]:
count =0
s = []
for d in gen_data:
    if (len(d['hyps'])) <= 1:
        count += 1
        s.append(d)

print(f'{count}')
for a in s:
    print(f"{a}\n")

In [ ]:
num_no50 = 0
for d in gen_data:
    if (len(d['token']) != 50):
        num_no50 += 1
        print(f"{d['name']} is {len(d['token'])}")
print(num_no50)

In [ ]:
num_no50 = 0
for d in gen_data:
    if (len(d['pll']) != len(d['token'])):
        print(f"illegal:{d['name']}")
    if (len(d['pll']) != 50):
        num_no50 += 1
        print(f"{d['name']} is {len(d['pll'])}")
print(num_no50)

In [ ]:
del gen_data

In [ ]:
split_num = 16
print(len(data['utts'].keys()))
save_num = len(data['utts'].keys()) // 16
print(save_num)

In [ ]:
for d in data['utts'].keys():
    print(type(data['utts'][d]['output']))
    if (isinstance(data['utts'][d]['output'], dict)):
        print(f"{data['utts'][d]['output'].keys()}")
    print(f"{len(data['utts'][d]['output'])}")
    print(f"{data['utts'][d]['output'][0]}")
    break

In [ ]:
import jiwer
from tqdm import tqdm

In [ ]:
data_count = 0
file_count = 1

data_list = []
for d in tqdm(data['utts'].keys(), ncols = 100):
    hyps = []
    errs = []
    scores = []
    ref = data['utts'][d]['output'][0]["token"].replace("<eos>", "")
    if ('am_score' in data['utts'][d]['output'][0].keys()):
        am_scores = []
    else:
        am_scores = None
    
    if ('ctc_score' in data['utts'][d]['output'][0].keys()):
        ctc_scores = []
    else:
        ctc_scores = None

    if ('lm_score' in data['utts'][d]['output'][0].keys()):
        lm_scores = []
    else:
        lm_scores = None
    
    for hyp in (data['utts'][d]['output']):
        hyp_token = hyp['rec_token'].replace("<eos>", "")
        hyp_token = hyp_token.replace(" ", "")
        hyp_token = " ".join([token for token in hyp_token])

        if (am_scores is not None):
            am_scores.append(hyp['am_score'])
        if (ctc_scores is not None):
            ctc_scores.append(hyp['ctc_score'])
        if (lm_scores is not None):
            lm_scores.append(hyp['lm_score'])

        scores.append(hyp['score'])

        measures = jiwer.compute_measures(ref, hyp_token)

        errs.append(
            {
                'err': measures['wer'],
                'hit': measures['hits'],
                'sub': measures['substitutions'],
                'del': measures['deletions'],
                'ins': measures['insertions'],
            }
        )

        hyps.append(hyp_token)
    
    data_list.append(
        {
            'name': d,
            'hyps': hyps,
            'am_score': am_scores,
            'ctc_score': ctc_scores,
            "lm_score": None if lm_scores is None else lm_scores,
            "score": scores,
            'err': errs,
            'ref': ref.lower()
        }
    )


    data_count += 1

    if (data_count % save_num == 0 and data_count > 0 and file_count < 16):
        with open(f'/mnt/disk6/Alfred/Rescoring/data/aishell2/raw/withLM/train/data_{file_count}.json', 'w') as f:
            json.dump(data_list, f, ensure_ascii = False, indent = 1)
        data_list = []
        data_count = 0
        print(f'file saved:/mnt/disk6/Alfred/Rescoring/data/aishell2/raw/withLM/train/data_{file_count}.json')
        file_count += 1

with open(f'/mnt/disk6/Alfred/Rescoring/data/aishell2/raw/withLM/train/data_{file_count}.json', 'w') as f:
    print(f'file saved:/mnt/disk6/Alfred/Rescoring/data/aishell2/raw/withLM/train/data_{file_count}.json')
    json.dump(data_list, f, ensure_ascii = False, indent = 1)

In [ ]:
combine_data = []
for i in range(1, 17):
    print(f'i:{i}')
    with open(f'/mnt/disk6/Alfred/Rescoring/data/aishell2/raw/withLM/train/data_{i}.json') as f:
        data_json = json.load(f)
        combine_data += data_json

with open(f'/mnt/disk6/Alfred/Rescoring/data/aishell2/data/withLM/train/data.json', 'w') as f:
     json.dump(combine_data, f, ensure_ascii = False, indent = 1)

In [ ]:
for i in range(1, 17):
    print(f'i:{i}')
    with open(f'/mnt/disk6/Alfred/Rescoring/data/aishell2/raw/withLM/train/data_{i}.json') as f:
        data_json = json.load(f)
        for data in data_json:
            if (data['name'] == 'IC9018W0502'):
                print(i)